In [17]:
from utils import read_config, OracleAgent, MySQLAgent
import re
import os
from py2neo import Graph, Node, Relationship
from langchain.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.chat_models  import AzureChatOpenAI

import google.generativeai as genai
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)


## DW

In [18]:
from utils import read_config, OracleAgent, MySQLAgent
import os
configs = read_config(".env/info.json")

os.environ["GOOGLE_API_KEY"] = configs['gkey']

O_KEY = configs['okey']
os.environ["OPENAI_API_KEY"] = O_KEY
DWDB = configs['DW_conn_info']
dw_agent = OracleAgent(DWDB)

In [19]:
mysql_configs = configs["Crawler_mysql_conn_info_BIData"]
mysql_agent = MySQLAgent(mysql_configs)

aoai_configs = configs['AOAI']

In [20]:
query = """
    SELECT view_name, text FROM ALL_Views
    where owner = 'YFYDW'
"""

view_info = dw_agent.read_table(query=query)
view_info['data_source'] = view_info['text'].apply(lambda x: re.search(r'FROM\s+(.*)', x, re.DOTALL | re.IGNORECASE).group(0))
view_info

,view_name,text,data_source
0,WACES_CARBON_EMS01_DF_V,"SELECT\n ORG.ORG_NAME,\n CHECKTYPE_CODE,...","FROM WACES_CARBON_EMS_DF F,\n DIM_ORG O..."
1,WACES_CARBON_EMS02_MF_V,"select \n org.org_name,\n c.check_date p...","from WBIMF_PRODUCTION_INDS_MF p,\n (sele..."


In [21]:
OPENAI_API_BASE = aoai_configs['OPENAI_API_BASE']
OPEN_AI_VERSION = aoai_configs['OPEN_AI_VERSION']
GPT_DEPLOYMENT_NAME = aoai_configs['GPT_DEPLOYMENT_NAME']
OPENAI_API_KEY = aoai_configs['OPENAI_API_KEY']
OPENAI_API_TYPE = aoai_configs['OPENAI_API_TYPE']

llm = AzureChatOpenAI(
    azure_endpoint=OPENAI_API_BASE,
    openai_api_version=OPEN_AI_VERSION,
    azure_deployment=GPT_DEPLOYMENT_NAME,
    openai_api_key=OPENAI_API_KEY,
    openai_api_type="azure",
)

# llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key = O_KEY)

In [29]:
system_template = """
    I will provide the table_name and the corresponing Oracle SQL. 
    Recongize all the unique identifer such as DIM_ORG org, the unique identifier is org.
    Understand the relationship, such as org.org_code which means the column org_code in DIM_ORG.
    Return the answer with the ORIGINAL table name instead of the unique identifier.

    Following is the example, give me the answer based on real case:
    ### Oracle SQL:
            from WBIPD_PRODUCTION_INDS_MF p,
            (select 
                    org_code,
                    check_date,
                    EMISSION_CATEGORY,
                    sum(co2_qty) co2_qty
                from WACES_CARBON_EMS_DF
                where 1=1
                and checktype_name = '月盤查'
                group by org_code, check_date,EMISSION_CATEGORY) c,
            DIM_ORG org
        where 1=1
        and c.org_code = p.org_code(+)
        and c.check_date = p.period_date(+)
        and org.org_code = c.org_code
        and org.rpt_used = 'ESG碳排分析'
    ### Result:

        Datasource = ['WACES_CARBON_EMS_DF', 'WBIPD_PRODUCTION_INDS_MF', 'DIM_ORG']

        Relationship = 
        [
        (GROUPBY: WACES_CARBON_EMS_DF filter with checktype_name = '月盤查', group by org_code, check_date,EMISSION_CATEGORY),
        (JOIN: WACES_CARBON_EMS_DF.org_code = WBIPD_PRODUCTION_INDS_MF.org_code(+)),
        (JOIN: WACES_CARBON_EMS_DF.period_date = WBIPD_PRODUCTION_INDS_MF.period_date(+)),
        (JOIN: DIM_ORG.org_code = WACES_CARBON_EMS_DF.org_code),
        (Filter: DIM_ORG.rpt_used = 'ESG碳排分析'),
        ]

    Just export the Datasource and Relationship part.
    
    table_name: {table_name}

    datasource: {datasource}

    """

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{table_name}, {datasource}")
]

CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [30]:
chain = CHAT_PROMPT | llm
input_data = {
    "table_name": view_info.iloc[0].view_name,
    "datasource": view_info.iloc[0].data_source
}
llm_response = chain.invoke(input_data)

In [31]:
llm_response.response_metadata['token_usage']

{'completion_tokens': 66, 'prompt_tokens': 581, 'total_tokens': 647}

In [33]:
llm_response.content

"### Result:\n\nDatasource = ['WACES_CARBON_EMS_DF', 'DIM_ORG']\n\nRelationship = \n[\n(JOIN: WACES_CARBON_EMS_DF.ORG_CODE = DIM_ORG.ORG_CODE),\n(Filter: DIM_ORG.RPT_USED = 'ESG碳排分析')\n]"

In [34]:
print(llm_response.content)

### Result:

Datasource = ['WACES_CARBON_EMS_DF', 'DIM_ORG']

Relationship = 
[
(JOIN: WACES_CARBON_EMS_DF.ORG_CODE = DIM_ORG.ORG_CODE),
(Filter: DIM_ORG.RPT_USED = 'ESG碳排分析')
]


In [35]:
import ast

def extract_lists(input_string):
    # Split the input string into sections based on the prefixes
    parts = input_string.split('\n\n')
    
    # Initialize dictionaries to hold the lists
    extracted_data = {}
    
    # Loop through each part and extract the corresponding list
    for part in parts:
        if part.startswith("Datasource"):
            # Extract the Datasource list using the prefix
            datasource_section = part[len("Datasource = "):].strip()
            extracted_data["Datasource"] = ast.literal_eval(datasource_section)
        elif part.startswith("Relationship"):
            # Extract the Relationship list using the prefix
            relationship_section = part[len("Relationship = "):].strip()
            # Remove brackets and split into list
            relationship_section = relationship_section.strip("[]\n")
            relationship_list = [item.strip().replace("(", "").replace(")", "").replace(",", "") for item in relationship_section.split(',\n')]
            extracted_data["Relationship"] = relationship_list
    
    return extracted_data

# Example usage
input_string = llm_response.content
result = extract_lists(input_string)

# Output the lists
datasource_list = result.get("Datasource", [])
relationship_list = result.get("Relationship", [])

print("Datasource List:", datasource_list)
print("Relationship List:", relationship_list)


Datasource List: ['WACES_CARBON_EMS_DF', 'DIM_ORG']
Relationship List: ['JOIN: WACES_CARBON_EMS_DF.ORG_CODE = DIM_ORG.ORG_CODE', "Filter: DIM_ORG.RPT_USED = 'ESG碳排分析'"]


### Export data into MySQL

In [31]:
view_lineage = view_info.copy()
view_lineage['lineage'] = ""
view_lineage.at[0, 'lineage'] = llm_response.content

In [32]:
view_lineage

,view_name,text,data_source,lineage
0,WACES_CARBON_EMS01_DF_V,"SELECT\n ORG.ORG_NAME,\n CHECKTYPE_CODE,...","FROM WACES_CARBON_EMS_DF F,\n DIM_ORG O...","Datasource = ['WACES_CARBON_EMS_DF', 'DIM_ORG'..."
1,WACES_CARBON_EMS02_MF_V,"select \n org.org_name,\n c.check_date p...","from WBIMF_PRODUCTION_INDS_MF p,\n (sele...",


#### add one complicated case

In [34]:
# rela = """
# Datasource = [
#     'apps.mtl_material_transactions',
#     'apps.oe_order_headers_all',
#     'apps.mtl_transaction_types',
#     'org_organization_definitions',
#     'apps.oe_order_lines_all',
#     'apps.oe_transaction_types_tl',
#     'apps.mtl_system_items_b',
#     'apps.mtl_item_categories_v',
#     'apps.ar_customers_all_v',
#     'xxom_quote_margin',
#     'xxinv_items_upload_t1',
#     'apps.ra_customer_trx_all',
#     'apps.ra_customer_trx_lines_all',
#     'apps.ra_cust_trx_types_all',
#     'apps.hz_cust_accounts',
#     'xxtcpq_pcg_avcost',
#     'hr_operating_units'
# ]\n\n
# Relationship = [
#     (JOIN: apps.oe_order_headers_all.header_id = apps.oe_order_lines_all.header_id),
#     (JOIN: apps.mtl_material_transactions.source_line_id = apps.oe_order_lines_all.line_id),
#     (JOIN: apps.mtl_material_transactions.transaction_type_id = apps.mtl_transaction_types.transaction_type_id),
#     (JOIN: org_organization_definitions.organization_id = apps.oe_order_headers_all.org_id),
#     (JOIN: org_organization_definitions.organization_id = apps.mtl_system_items_b.organization_id),
#     (JOIN: apps.mtl_system_items_b.inventory_item_id = apps.oe_order_lines_all.inventory_item_id),
#     (JOIN: apps.mtl_system_items_b.inventory_item_id = apps.mtl_item_categories_v.inventory_item_id),
#     (JOIN: apps.mtl_item_categories_v.organization_id = org_organization_definitions.organization_id),
#     (JOIN: apps.ar_customers_all_v.customer_id = apps.oe_order_headers_all.sold_to_org_id),
#     (JOIN: apps.hz_cust_accounts.cust_account_id = apps.ar_customers_all_v.customer_id),
#     (JOIN: apps.ra_customer_trx_lines_all.customer_trx_id = apps.ra_customer_trx_all.customer_trx_id),
#     (JOIN: apps.ra_customer_trx_lines_all.inventory_item_id = apps.mtl_system_items_b.inventory_item_id),
#     (JOIN: apps.ra_cust_trx_types_all.cust_trx_type_id = apps.ra_customer_trx_lines_all.cust_trx_type_id),
#     (JOIN: apps.ra_cust_trx_types_all.org_id = org_organization_definitions.operating_unit),
#     (JOIN: xxtcpq_pcg_avcost.ebs_so_header_id = apps.oe_order_headers_all.header_id),
#     (JOIN: xxom_quote_margin.quote_number = apps.oe_order_headers_all.order_number),
#     (JOIN: xxinv_items_upload_t1.segment1 = apps.mtl_system_items_b.segment1),
#     (Filter: apps.mtl_item_categories_v.category_set_name = 'YFY Inventory'),
#     (Filter: apps.mtl_item_categories_v.enabled_flag = 'Y'),
#     (Filter: apps.oe_transaction_types_tl.LANGUAGE = 'US'),
#     (Filter: org_organization_definitions.organization_code NOT IN ('AEG')),
#     (Filter: apps.oe_order_lines_all.item_type_code <> 'MODEL'),
#     (Filter: apps.oe_order_lines_all.flow_status_code = 'CLOSED'),
#     (Filter: apps.mtl_material_transactions.subinventory_code = 'STAGE')
# ]

# """


In [35]:
# view_lineage.at[2, 'view_name'] = 'XXBI_SALES_DETAILS_PCG_TW'
# view_lineage.at[2, 'text'] = text_sql
# view_lineage.at[2, 'lineage'] = rela


In [36]:
# mysql_agent.write_table(data=view_lineage, table_name='data_lineage', if_exists='replace', data_type=None, index=None)

## BIDB

In [39]:
BIDB = configs['BIDB_conn_info']
bidb_agent = OracleAgent(BIDB)

In [78]:
query = """
SELECT view_name, text FROM ALL_Views
where owner = 'ODS' AND rownum <= 5
"""

bidb_view = bidb_agent.read_table(query=query)

In [96]:
bidb_view['text'] = bidb_view['text'].str.replace('\n', ' ')
bidb_view['text'] = bidb_view['text'].str.replace('\t', ' ')
bidb_view['source'] = bidb_view['text'].apply(lambda x: re.search(r'FROM\s+(.*)', x, re.DOTALL | re.IGNORECASE).group(0))
bidb_view['source'] = bidb_view['source'].apply(lambda x: x.split('UNION')[0].strip())
bidb_view['source'] = bidb_view['source'].apply(lambda x:re.sub(r'--.*?(\s+|$)', '', x)) 


In [97]:
bidb_view

,view_name,text,source
0,C$_0W_YFY_AV_TW_R,"select ""C1_ROW_ID"",""C2_ORG_ID"",""C3_REF_AV_HEAD...",from ( select W_YFY_AV_TW_R.ROW_ID C1_ROW...
1,C$_0W_YFY_IND_FIN_INFO_FS,"select FIN_INFO.SEQ C1_SEQ, FIN_INFO.ACC...",from ODS.TC$_0W_YFY_IND_FIN_INFO_FS FIN_INFO
2,OP_FACT_CHP_INVENTORY_ETH_PULP,"SELECT PERIOD_NAME,STOCK_DATE TDATE,ORG_CODE,'...",FROM W_FACTORY_INV_BALANCE_F
3,OP_FACT_CHP_INVENTORY_REDEFINE,"SELECT PERIOD_NAME,TDATE,ORG_CODE ,CASE ORG_...","FROM ( SELECT F.PERIOD_NAME,F.BALANCE_DATE T..."
4,OP_FACT_CHP_SALES_DETAILS,"SELECT '當月受訂' TYPE, --GREATEST(F.REQU...","from W_CHP_SALES_NOMANUAL_F F ,W_CHP_ORDER..."


In [115]:
for row in bidb_view.itertuples():
    print(row.source)

from (    select   W_YFY_AV_TW_R.ROW_ID C1_ROW_ID,  W_YFY_AV_TW_R.ORG_ID C2_ORG_ID,  W_YFY_AV_TW_R.REF_AV_HEADER_ID C3_REF_AV_HEADER_ID,  W_YFY_AV_TW_R.ORDER_AV C4_ORDER_AV,  W_YFY_AV_TW_R.UPDATE_DT C5_UPDATE_DT,  W_YFY_AV_TW_R.CREATION_DT C6_CREATION_DT,  W_YFY_AV_TW_R.DATASOURCE_NUM_ID C7_DATASOURCE_NUM_ID,  W_YFY_AV_TW_R.MODEL_ITEM C8_MODEL_ITEM,  W_YFY_AV_TW_R.ORDER_TRANSPORT_FEE C9_ORDER_TRANSPORT_FEE,  W_YFY_AV_TW_R.PRICECLASS_COST C10_PRICECLASS_COST from ODS.W_YFY_AV_TW_R   W_YFY_AV_TW_R where (1=1)        )
from ODS.TC$_0W_YFY_IND_FIN_INFO_FS  FIN_INFO
FROM W_FACTORY_INV_BALANCE_F
FROM (   SELECT F.PERIOD_NAME,F.BALANCE_DATE TDATE   ,NVL(R.ORG_CODE_NEW,O.ORG_CODE) ORG_CODE   ORG_CODE_Original   ,F.INVENTORY_ITEM_ID   ITEM_NO   ,'MT' UOM,F.END_QTY / 1000 AS QTY   FROM W_YFY_INV_BALANCE_F F       ,w_chp_item_d I       ,W_YFY_ORG_D O       ,W_CHP_TS_BELONG_ORG_R R   WHERE 1=1     AND F.ORGANIZATION_ID IN (290,291,305,286,287,288,289)     AND F.ORGANIZATION_ID = I.ORGANIZATION_ID 